In [ ]:
%matplotlib inline


from beras.data_utils import HDF5Tensor
import matplotlib.pyplot as plt
import numpy as np
from deepdecoder.utils import masks, visualise_tiles, zip_visualise_tiles, np_binary_mask
from deepdecoder.dcgan_train import generator, discriminator, dcmogan
from deepdecoder.adaptive_sampling import curriculum_grids_generator, curriculum_grids
from beesgrid import MASK, CONFIG_ROTS, CONFIG_RADIUS, CONFIG_CENTER, TAG_SIZE
from keras.optimizers import SGD, Adam, RMSprop
import importlib
import h5py
import pylab
import time

pylab.rcParams['figure.figsize'] = (18, 18)

In [ ]:
batch_size = 128
def grid_generator(hardness, batch_size, input_dim=40):
    while True:
        params, grid_idx = curriculum_grids(hardness, batch_size)
        bw_grid = np_binary_mask(grid_idx)
        z = np.random.uniform(-1, 1, (params.shape[0], input_dim - params.shape[1]))
        yield np.concatenate([params, z], axis=1), bw_grid
        
input_dim = 40
params, grids = next(grid_generator(0.1, batch_size, input_dim))
print(params.shape)
assert params.shape[1] == input_dim

In [ ]:
g = generator(input_dim=40)

In [ ]:
start = time.time()
g.compile(Adam(lr=0.003), 'mse')
print("Compiling done in {}s".format(time.time() - start))

In [ ]:
def visualise_g(h):
    params, grids = next(grid_generator(h, batch_size))
    pred_grids = g.predict(params)
    zip_visualise_tiles(grids, pred_grids)

In [ ]:
h = 0.25
g.fit_generator(grid_generator(h, batch_size), samples_per_epoch=100*batch_size, nb_epoch=10, verbose=1)
visualise_g(h)

In [ ]:
h = 0.3
g.fit_generator(grid_generator(h, batch_size), 
                samples_per_epoch=100*batch_size, 
                nb_epoch=10, verbose=1)
visualise_g(h)

In [ ]:
h = 0.6
g.fit_generator(grid_generator(h, batch_size), 
                samples_per_epoch=100*batch_size, 
                nb_epoch=1, verbose=1)
visualise_g(h)

In [ ]:
h = 0.85
g.fit_generator(grid_generator(h, batch_size), 
                samples_per_epoch=100*batch_size,
                nb_epoch=1, verbose=1)
visualise_g(h)

In [ ]:
h = 1.0
g.fit_generator(grid_generator(h, batch_size),     
                samples_per_epoch=100*batch_size, 
                nb_epoch=3*60, verbose=1)
visualise_g(h)

In [ ]:
visualise_g(h)